## Homework: Agents

### Preparation

In [1]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description

In [6]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get weather data by city name",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "city name"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [3]:
! wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-16 13:58:20--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py’

chat_assistant.py   100%[===================>]   3.40K  --.-KB/s    in 0s      

2025-07-16 13:58:20 (51.2 MB/s) - ‘chat_assistant.py’ saved [3485/3485]



In [31]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data by city name',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string', 'description': 'city name'}},
   'required': ['city'],
   'additionalProperties': False}}]

In [9]:
from openai import OpenAI
client = OpenAI()

In [35]:
developer_prompt = """
You're an assistant. 
You're given a question about weather in some city and your task is to answer it.

Use function provided to answer the question.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [20]:
chat.run()

You: weather in Tbilisi?


You: and in Batumi?


You: and Berlin?


You: and Tbilisi again?


You: and Berlin?


You: what is your favorite color?


You: temperature in Rome


You: stop


Chat ended.


### Q2. Adding another tool

In [32]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [33]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set weather data by city name",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "city name"
            },
            "temp": {
                "type": "number",
                "description": "temperature value"
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [34]:
tools.add_tool(set_weather, set_weather_tool)
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get weather data by city name',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string', 'description': 'city name'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Set weather data by city name',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string', 'description': 'city name'},
    'temp': {'type': 'number', 'description': 'temperature value'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [37]:
chat.run()

You: temp in Batumi


You: and Tbilisi


You: stop


Chat ended.


### Q3. Install FastMCP

In [38]:
! pip list | grep fastmcp

fastmcp                   2.10.5


### Q4. Simple MCP Server

Starting MCP       server.py:1371
                             server 'Demo 🚀'                 
                             with transport                   
                             'stdio'

### Q5. Protocol

In [39]:
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}

{'jsonrpc': '2.0',
 'id': 3,
 'method': 'tools/call',
 'params': {'name': 'get_weather', 'arguments': {'city': 'Berlin'}}}

### Q6. Client

In [59]:
import weather_server
from fastmcp import Client

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # List the known tools (optional, helpful for debugging)
        tools = await mcp_client.list_tools()
        print(tools)

In [60]:
await main()

[Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Te

In [61]:
import weather_server
from fastmcp import Client

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # Call the 'get_weather' tool
        result = await mcp_client.call_tool(
            "get_weather",
            {"city": "Berlin"}
        )
        print("Weather result:", result)

In [62]:
await main()

Weather result: CallToolResult(content=[TextContent(type='text', text='20.0', annotations=None, meta=None)], structured_content={'result': 20.0}, data=20.0, is_error=False)
